# Artificial Intelligence Project

## Group 1:
### - Rajoelisoa Enorian
### - Hirli Baptiste
### - Bhattacharjee Ankit
### - Caumartin Evan

# Data Preprocessing

### 1. Load the data
### 2. Merge the data
### 3. Calculate the average working time
### 4. Remove the outliers
### 5. Drop the unnecessary columns
### 6. Encode the categorical data
### 7. Impute the missing values
### 8. Scale the data
### 9. Save the data

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

In [ ]:
full_employee_data: pd.DataFrame = pd.DataFrame()

is_ethic = True
remove_outliers = False

# Load the data

### Load the data from the datasets folder

We read multiple CSV files into pandas DataFrames. The general_data, employee_survey_data, and manager_survey_data DataFrames contain general, employee survey, and manager survey data, respectively. The in_time and out_time DataFrames contain timestamp data for employee check-in and check-out times, which are converted to datetime format.

In [ ]:
general_data = pd.read_csv('datasets/general_data.csv')
employee_survey_data = pd.read_csv('datasets/employee_survey_data.csv')
manager_survey_data = pd.read_csv('datasets/manager_survey_data.csv')
in_time: pd.DataFrame = pd.read_csv('datasets/in_time.csv', parse_dates=True)
out_time = pd.read_csv('datasets/out_time.csv', parse_dates=True)

# Merge the data

We are merging general data, employee survey data, and manager survey data into a single DataFrame.

In [ ]:
full_employee_data = general_data.merge(employee_survey_data, on='EmployeeID')
full_employee_data = full_employee_data.merge(manager_survey_data, on='EmployeeID')

# Calculate the average working time

We analyzed employee working patterns by calculating key metrics such as average arrival time, departure time, and working hours. Starting with arrival (`in_time`) and departure (`out_time`) datasets, we determined the total time spent at work each day by subtracting arrival times from departure times and converting the results into hours.
(Here, we assumed that the arrival and departure times were on the same day to simplify the analysis. If the data spanned multiple days, additional processing would be required to account for overnight shifts.)
We then calculated the **AverageArrivalTime** (mean of arrival times), **AverageDepartureTime** (latest departure time), and **AverageWorkingTime** (average daily working hours) for each employee. These insights were combined into a new dataset and merged with the general employee data to provide a comprehensive view, including details like name and department alongside their working time metrics.
For instance, an employee arriving at 9:15 AM and leaving at 6:30 PM consistently would have an average working time of 9 hours. This analysis helps in understanding employee work habits, identifying trends, and improving operational efficiency.

In [ ]:
# Fill missing values
in_time = in_time.fillna('1970-01-01 00:00:00')
out_time = out_time.fillna('1970-01-01 00:00:00')

# Convert all times to datetime
in_time.iloc[:, 1:] = in_time.iloc[:, 1:].apply(pd.to_datetime).apply(lambda x: x.dt.hour * 3600 + x.dt.minute * 60 + x.dt.second)
out_time.iloc[:, 1:] = out_time.iloc[:, 1:].apply(pd.to_datetime).apply(lambda x: x.dt.hour * 3600 + x.dt.minute * 60 + x.dt.second)

# Replace 0 with row medians using mask or where to avoid the FutureWarning
in_time.iloc[:, 1:] = in_time.iloc[:, 1:].apply(lambda row: row.mask(row == 0, row.median()), axis=1)
out_time.iloc[:, 1:] = out_time.iloc[:, 1:].apply(lambda row: row.mask(row == 0, row.median()), axis=1)

# Calculate working time for each day in hours
working_time = (out_time.iloc[:, 1:] - in_time.iloc[:, 1:])

# Average arrival time
average_arrival_time = in_time.iloc[:, 1:].mean(axis=1)

# Average departure time
average_departure_time = out_time.iloc[:, 1:].max(axis=1)

# Calculate average working time
average_working_time = working_time.mean(axis=1) / 3600

# Convert average times directly to total seconds
result = pd.DataFrame({
    'EmployeeID': in_time.iloc[:, 0],  # Use the first column directly
    'AverageArrivalTime': average_arrival_time.round(0).astype(int),
    'AverageDepartureTime': average_departure_time.round(0).astype(int),
    'AverageWorkingTime': average_working_time.round(2)
})

result.to_csv('working_time_data.csv', index=False)

# Merge with other employee data
full_employee_data = full_employee_data.merge(result, on='EmployeeID')

# Remove the outliers

We removed the outliers from the dataset using the Interquartile Range (IQR) method. Outliers are data points that significantly differ from other observations in the dataset. They can skew the results of statistical analyses and machine learning models.
The IQR method identifies outliers by calculating the range between the first and third quartiles of the data and flagging values that fall below the lower bound (Q1 - 1.5 * IQR) or above the upper bound (Q3 + 1.5 * IQR). Outliers are then replaced with the lower or upper bound values to ensure they do not impact the analysis.

In [ ]:
def find_outliers_iqr(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = (series < lower_bound) | (series > upper_bound)
    return outliers

def cap_outliers_in_dataframe(df, excluded_columns=None):
    df = df.copy()
    
    if not remove_outliers:
        return df

    for column in df.columns:
        if excluded_columns and column in excluded_columns:
            continue
        
        if np.issubdtype(df[column].dtype, np.number):  # Check if the column contains numeric data
            outliers = find_outliers_iqr(df[column])

            # Cap the outliers by replacing them with lower/upper bounds
            Q1 = df[column].quantile(0.25)
            Q3 = df[column].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            df[column] = np.where(outliers, np.clip(df[column], lower_bound, upper_bound), df[column])

    return df

In [ ]:
full_employee_data = cap_outliers_in_dataframe(full_employee_data, excluded_columns=['PerformanceRating'])

## Display the first 5 rows of the data

We displayed the first five rows of the dataset to understand its structure and contents. This step provides an overview of the data, including the column names, data types, and values. It helps in identifying any potential issues, such as missing values, outliers, or incorrect data types.

In [ ]:
full_employee_data.head()

## Display the shape of the data

We displayed the shape of the dataset to determine the number of rows and columns it contains. This information is essential for understanding the size and structure of the data, which can impact the analysis and modeling process.

In [ ]:
full_employee_data.describe()

## Display the columns of the data

We displayed the column names of the dataset to identify the variables or features available for analysis. Understanding the columns helps in selecting relevant data for specific tasks, such as predictive modeling, clustering, or classification.

In [ ]:
full_employee_data.isna().sum()

# Drop the unnecessary columns

We dropped the unnecessary columns from the dataset to focus on the relevant features for analysis. Removing redundant or irrelevant columns helps in reducing the dimensionality of the data and improving the performance of machine learning models.

The columns `EmployeeCount`, `Over18`, and `StandardHours` were dropped as they contained constant values for all employees and did not provide any useful information for analysis.

The `EmployeeID` column was also removed as it served as an identifier and was not required for further analysis.

In [ ]:
# EmployeeCount : All values are 1
# Over18 : All values are 'Y'
# StandardHours : All values are 8
columns_to_drop = ['EmployeeCount', 'Over18', 'StandardHours', 'EmployeeID']

# Drop additional columns if is_ethic is True
if is_ethic:
    columns_to_drop += ['Age', 'Education', 'MaritalStatus', 'Gender', 'EducationField']

full_employee_data = full_employee_data.drop(columns_to_drop, axis=1)

# Encode the categorical data

In this step, **we encoded categorical variables** in the employee dataset to prepare the data for machine learning models, which typically require numerical inputs. We targeted specific categorical columns: `Department`, `BusinessTravel`, `JobRole`, `MaritalStatus`, `Gender`, and `EducationField`.
For each of these columns, we used **one-hot encoding**, a technique that converts each unique category into separate binary columns. For example, if the `Department` column contains values like "Sales," "HR," and "R&D," one-hot encoding will create three new columns: `Department_Sales`, `Department_HR`, and `Department_R&D`. Each row will have a value of `1` in the relevant column and `0` in the others. This ensures that the categorical data is represented numerically without introducing any unintended ordinal relationships.

To maintain flexibility, we first checked if each column in the list still existed in the dataset before applying the encoding. This avoids errors if columns were previously removed or altered. After encoding, the original categorical columns were dropped from the dataset to avoid redundancy. In addition to encoding, we transformed the **`Attrition`** column, which indicates whether an employee has left the company, into binary values.
Specifically, we mapped `'Yes'` to `1` and `'No'` to `0`, making it suitable for classification models. This binary encoding enables the model to treat attrition as a target variable for predictive analysis. By the end of this process, all relevant categorical features and the target variable were represented numerically, ensuring the dataset was ready for further data processing or model training.

In [ ]:
#Encoding
cat_data = ['Department', 'BusinessTravel', 'JobRole', 'MaritalStatus', 'Gender', 'EducationField']

# Only encode columns that are still present in the dataframe
for i in cat_data:
    if i in full_employee_data.columns:
        vals = pd.get_dummies(full_employee_data[i], sparse=True)
        full_employee_data = pd.concat([full_employee_data, vals], axis=1)
        full_employee_data = full_employee_data.drop(i, axis=1)

# Map Attrition column to binary values
full_employee_data['Attrition'] = full_employee_data['Attrition'].map({'Yes': 1, 'No': 0})

# Impute the missing values

In this step, we **imputed missing values** in the dataset to ensure that the data was complete and ready for analysis. Missing values can occur due to various reasons, such as data entry errors, system failures, or incomplete records. Imputation is the process of filling in missing values with estimated or calculated values based on the available data.
We used the **SimpleImputer** class from scikit-learn to impute missing values in the dataset. We chose the **median strategy** to replace missing values with the median of each column. The median is a robust measure of central tendency that is less sensitive to outliers than the mean. By imputing missing values with the median, we aimed to maintain the integrity of the data and avoid introducing bias or distortion.
After imputing missing values, we checked the dataset for any remaining null values to confirm that the imputation process was successful. This ensured that the data was clean, complete, and ready for further analysis or modeling.

In [ ]:
#Imputation
imputer = SimpleImputer(strategy='median')

for cat_name in full_employee_data.columns:
    full_employee_data[cat_name] = imputer.fit_transform(full_employee_data[[cat_name]])

print(full_employee_data.isnull().sum())

# Scale the data

In this step, we **scaled the data** to ensure that all features were on a similar scale, which is essential for many machine learning algorithms. Scaling the data helps in improving the performance and convergence of models by ensuring that no single feature dominates the others.
We used the **MinMaxScaler** class from scikit-learn to scale the data to a specific range. Min-max scaling transforms the data to a specified range (default is 0 to 1) by subtracting the minimum value and dividing by the range. This normalization technique preserves the relationships between the data points while ensuring that all features are within the same scale.
After scaling the data, we checked the summary statistics of the dataset to confirm that all features were scaled appropriately. This step ensured that the data was ready for further analysis, modeling, or visualization.

In [ ]:
# Scaling
scaler = MinMaxScaler()

no_scale = ["PerformanceRating", "Attrition"]

for cat_name in full_employee_data.columns:
    if cat_name not in no_scale:
        full_employee_data[cat_name] = scaler.fit_transform(full_employee_data[[cat_name]])
    
print(full_employee_data.head())

In [ ]:
# Display the first 5 rows of the data
print(full_employee_data.head())

# Display the shape of the data
print(full_employee_data.shape)

# Display the columns of the data
print(full_employee_data.columns)

# Display the summary statistics of the data
print(full_employee_data.describe(include='all'))

# Display the missing values in the data
print(full_employee_data.isnull().sum())

# Display the unique values in the data
full_employee_data.hist(bins=50, figsize=(20, 15))
plt.show()

In [ ]:
def display_correlation_matrix(data: pd.DataFrame):
    # Display the correlation matrix of the data
    corr_matrix = data.corr()

    # Create a heatmap of the correlation matrix
    plt.figure(figsize=(20, 15), edgecolor='white')
    sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='YlGnBu', cbar=False)
    plt.show()

display_correlation_matrix(full_employee_data)

# Remove the columns with high correlation

We removed the columns with high correlation to avoid multicollinearity, which can lead to unstable model coefficients and inaccurate predictions. Multicollinearity occurs when two or more independent variables are highly correlated, making it difficult to determine the individual effect of each variable on the target variable.
We identified the highly correlated columns using the correlation matrix and removed them from the dataset. This step helped in improving the model's performance and interpretability by eliminating redundant features and reducing the complexity of the data.
We removed the columns `Travel_Rarely`, `AverageDepartureTime`, `YearsWithCurrManager`, `Sales` and `PerformanceRating` as they were highly correlated with other features in the dataset.

In [ ]:
# Remove the columns with high correlation
full_employee_data = full_employee_data.drop("Travel_Rarely", axis=1)
full_employee_data = full_employee_data.drop("AverageDepartureTime", axis=1)
full_employee_data = full_employee_data.drop("YearsWithCurrManager", axis=1)
full_employee_data = full_employee_data.drop("Sales", axis=1)
full_employee_data = full_employee_data.drop("PerformanceRating", axis=1)

display_correlation_matrix(full_employee_data)

In [ ]:
full_employee_data.to_csv('full_employee_data_cleaned.csv', index=False)

# Testing Different ML Models

#### *Linear Kernel Classification*

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from models.statistics_helper import StatisticsHelper

# Load the data
full_employee_data = pd.read_csv('./full_employee_data_cleaned.csv')

# Prepare the data
target_column = 'Attrition'
X = full_employee_data.drop(target_column, axis=1)
y = full_employee_data[target_column]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

lr_model = LinearSVC(random_state=42, dual='auto')
lr_model.fit(X_train, y_train)

# Make predictions
y_pred = lr_model.predict(X_test)

# Display the statistics
stats_helper = StatisticsHelper(X, y, lr_model, y_test, y_pred)
lr_acc = stats_helper.ret_cross_val_score()
lr_roc = stats_helper.ret_roc_auc_score()
stats_helper.show_all()

#### *Logistic Regression*

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg_model = LogisticRegression(random_state=42, max_iter=1000)  # Increase max_iter if convergence warnings occur
log_reg_model.fit(X_train, y_train)

# Make predictions
y_pred = log_reg_model.predict(X_test)

# Display the statistics
stat_helper = StatisticsHelper(X, y, log_reg_model, y_test, y_pred)
log_acc = stats_helper.ret_cross_val_score()
log_roc = stats_helper.ret_roc_auc_score()
stat_helper.show_all()


#### *Decison Tree*

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier(criterion='gini', random_state=42)
dt_model.fit(X_train, y_train)

# Make predictions
y_pred = dt_model.predict(X_test)

# Display the statistics
stats_helper = StatisticsHelper(X_train, y_train, dt_model, y_test, y_pred)
dt_acc = stats_helper.ret_cross_val_score()
dt_roc = stats_helper.ret_roc_auc_score()
stats_helper.show_all()


#### *Random Forest*

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=1000, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)

# Display the statistics
stats_helper = StatisticsHelper(X_train, y_train, rf_model, y_test, y_pred)
rf_acc = stats_helper.ret_cross_val_score()
rf_roc = stats_helper.ret_roc_auc_score()
stats_helper.show_all()


#### *K-Nearest-Neighbor*

##### Testing for the optimal n_neighbors value to use in this case

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

#Test for the best n_neighbors value
for i in range(1,20):
    knn_model = KNeighborsClassifier(n_neighbors=i)
    knn_model.fit(X_train, y_train)

    #Make predictions
    y_pred = knn_model.predict(X_test)
    stats_helper = StatisticsHelper(X, y, knn_model, y_test, y_pred)
    print(f"N = {i}")
    stats_helper.show_accuracy()
    stats_helper.show_cross_val_score()
    stats_helper.show_roc_auc_score()

##### It can be seen from the results above that N = 1,2,3 has the best results but is not reliable in a more varied dataset as it basically just picks the number next to it. N = 7 appears to reach a good result among the rest so that is that we will be using.

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=7)
knn_model.fit(X_train, y_train)

# Make predictions
y_pred = knn_model.predict(X_test)

stats_helper = StatisticsHelper(X, y, knn_model, y_test, y_pred)
knn_acc = stats_helper.ret_cross_val_score()
knn_roc = stats_helper.ret_roc_auc_score()
stats_helper.show_all()

#### *Support Vector Machine (Non-Linear)*

##### Performing Grid Search will help us find optimal parameters to be used for the Support Vector Machine (SVM)

In [ ]:
from sklearn.model_selection import GridSearchCV 
from sklearn.svm import SVC

param_grid = {
    'C': [1, 10],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'gamma': [0.01, 0.1]
}

svm = SVC(random_state=42)

grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5, verbose=1, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Display the best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

##### The parameters give above were chosen after some trial and error. C values below 1 and above 10 were insignificant or redundent. Gamma values below 0.01 would lead to minimal learning of the negatives and above 0.1 would lead to clear overfitting. Hence among the given values, the best model was found.

In [ ]:
svm_model = grid_search.best_estimator_
svm_model.fit(X_train, y_train)

# Make predictions
y_pred = svm_model.predict(X_test)

stats_helper = StatisticsHelper(X, y, svm_model, y_test, y_pred)
svm_acc = stats_helper.ret_cross_val_score()
svm_roc = stats_helper.ret_roc_auc_score()
stats_helper.show_all()

#### *Simple Neural Network*

In [ ]:
from keras import Sequential
from keras.api.layers import Dense

nn_model = Sequential([
    Dense(128, activation="relu"),
    Dense(32, activation="relu"),
    Dense(1, activation="sigmoid"),
])

#### We are using standard hyperparameters for this model, using the loss function of binary cross entropy since it fits this situation. The model has 3 layers of neurons, with the final layer having a single output with the sigmoid function.

In [ ]:
nn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#### The number of epochs here was chosen where the accuracy was sufficiently high. Increasing the number of epochs further makes the model perfectly accurate, however it would certainly be overfitted. Hence to avoid that, it has been kept at a sufficient level for this case.

In [ ]:
history = nn_model.fit(X_train, y_train, epochs=50)

#### Formatting the float values to their respective integer class values using a threshold

In [ ]:
y_pred = nn_model.predict(X_test)
for i in range(len(y_pred)):
    if y_pred[i] < 0.5:
        y_pred[i] = 0
    else:
        y_pred[i] = 1
y_pred

In [ ]:
stats_helper = StatisticsHelper(X, y, nn_model, y_test, y_pred)
nn_acc = stats_helper.ret_accuracy()
nn_roc = stats_helper.ret_roc_auc_score()
stats_helper.show_accuracy()
stats_helper.show_roc_auc_score()
stats_helper.show_classification_report()
stats_helper.show_confusion_matrix()

### Model Comparison

In [ ]:
# Comparing models

models = ['Linear Kernal Classification', 'Logistic Regression', 'Decision Tree Classifier', 'Random Forest Classifier', 'K-Nearest_Neighbor Classifier', 'Support Vector Machine (Non-Linear)', 'Neural Network']
res_vals = [[lr_acc, log_acc, dt_acc, rf_acc, knn_acc, svm_acc, nn_acc],
            [lr_roc, log_roc, dt_roc, rf_roc, knn_roc, svm_roc, nn_roc]]

res_df = pd.DataFrame(res_vals, columns=models, index=['Accuracy', 'ROC_AUC'])

import seaborn as sns
sns.heatmap(res_df, annot=True, cmap='Blues', fmt='.3g')

### From these results, it is clear that Random Forest Classifier and Neural Network give the best results.

# Conclusion

The analysis of employee attrition at HumanForYou revealed critical factors influencing turnover, such as job satisfaction, work-life balance, frequent business travel, and tenure. Employees with low satisfaction scores, higher commute distances, or limited career growth opportunities showed a significantly higher likelihood of leaving the company. Additionally, certain job roles and hierarchical levels exhibited disproportionately high attrition rates, indicating specific areas requiring targeted interventions.

To address these challenges, the company should focus on improving work-life balance and job satisfaction through initiatives such as flexible schedules, enhanced benefits, and professional development programs. Leveraging predictive models can enable the identification of at-risk employees, allowing proactive retention strategies. By adopting these measures, HumanForYou can reduce attrition, enhance project delivery timelines, and optimize resource allocation for recruitment and training.

By leveraging predictive analytics, the company can identify employees most at risk of leaving and implement targeted retention strategies. These efforts, coupled with fostering a culture of continuous improvement, will not only reduce turnover but also strengthen employee morale and productivity. With these measures, HumanForYou can ensure the stability of its workforce, enhance project delivery, and build a strong reputation as a desirable workplace in the competitive pharmaceutical industry.